In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.ollama import OllamaEmbeddings
# from langchain_fireworks import FireworksEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [6]:
vector_store=Chroma.from_documents(documents,OllamaEmbeddings(model="nomic-embed-text"))

In [7]:
retriever=vector_store.as_retriever()

In [8]:
from langchain.tools.retriever import create_retriever_tool

In [9]:
retriever_tool=create_retriever_tool(retriever,"langsmith_search","search for information about langsmith")

In [10]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [11]:
arxiv.name

'arxiv'

In [12]:
tools=[retriever_tool,wiki,arxiv]

In [13]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    api_key="ollama",
    model="llama3",
    base_url="http://localhost:11434/v1",
)

In [34]:
from langchain import hub

prompt=hub.pull("vipintomar0/superb_system_instruction_prompt")

In [35]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [36]:
from langchain.agents import create_tool_calling_agent
# from langchain.agents import create_openai_tools_agent

agent=create_tool_calling_agent(model,tools,prompt)
# agent=create_openai_tools_agent(model,tools,prompt)

In [37]:
# agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [38]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'vipintomar0', 'lc_hub_repo': 'superb_system_instruction_prompt', 'lc_hub_commit_hash': 

In [43]:
agent_executor.invoke({"input":"what is LangSmith"})



> Entering new AgentExecutor chain...
LangSmith!

LangSmith is an AI-powered language processing tool that helps you with writing, grammar, and syntax checks. It's designed to be a smart editor that provides suggestions and feedback on your written content to make it clearer, more concise, and free of errors.

Some of the key features of LangSmith include:

1. Grammar and Syntax Checks: LangSmith detects grammatical errors, punctuation mistakes, and syntax issues in your text and suggests corrections.
2. Sentence Structure Analysis: It analyzes the structure of your sentences, providing suggestions on how to improve sentence length, clarity, and flow.
3. Style Guidance: LangSmith provides suggestions on style, tone, and voice to help you refine your writing.
4. Contextual Suggestions: It takes into account the context in which your text will be used (e.g., blog post, academic paper, social media) to offer targeted suggestions.
5. Multi-Format Support: LangSmith supports various forma

{'input': 'what is LangSmith',
 'output': "LangSmith!\n\nLangSmith is an AI-powered language processing tool that helps you with writing, grammar, and syntax checks. It's designed to be a smart editor that provides suggestions and feedback on your written content to make it clearer, more concise, and free of errors.\n\nSome of the key features of LangSmith include:\n\n1. Grammar and Syntax Checks: LangSmith detects grammatical errors, punctuation mistakes, and syntax issues in your text and suggests corrections.\n2. Sentence Structure Analysis: It analyzes the structure of your sentences, providing suggestions on how to improve sentence length, clarity, and flow.\n3. Style Guidance: LangSmith provides suggestions on style, tone, and voice to help you refine your writing.\n4. Contextual Suggestions: It takes into account the context in which your text will be used (e.g., blog post, academic paper, social media) to offer targeted suggestions.\n5. Multi-Format Support: LangSmith supports 

In [ ]:
agent_executor.invoke({"input":"tell me about machine learning"})